In [1]:
import pandas as pd
origin = pd.read_excel('./filename')  # 데이터 입력
origin = origin.iloc[:,2]   # 발화가 있는 셀을 참조

import re
def def_pre(origin):
    dialogue_list = []  
    for sentece in origin:
        sentece = re.sub(r'\([^)]*\)', '', str(sentece))
        sentece = re.sub('[.,?/*!"]', '', str(sentece))
        if sentece[:1] =='아':
            dialogue_list.append(sentece[2:])
    return dialogue_list
         
def def_remove_space(dialogue_list):
    strip_senten = []
    for i in dialogue_list:
        strip_senten.append(i.strip())
    return strip_senten
    
def def_remove_space2(strip_senten):
    full_senten = []  
    for i in range(len(strip_senten)):
        if( len(strip_senten[i]) > 0):
            full_senten.append(strip_senten[i])
        else:
            continue
    return full_senten

from konlpy.tag import Kkma
kkma = Kkma()

def def_kkmaNoun(full_senten):
    origin_noun_kkma = []
    for i in range(len(full_senten)):
        nouns = kkma.pos(full_senten[i])
        # print(nouns)
        etc = []
        for word in nouns:
            if (len(word[0])>1) and (word[1][0] == 'N'):
                etc.append(word[0])
            else:
                continue
        origin_noun_kkma.append(etc)
    return origin_noun_kkma

def bind_document(documents):
    new_documents=[]
    for i in range(len(documents)-2):
        new_documents.append(documents[i]+ ' ' + documents[i+1]+ ' ' + documents[i+2])
    return new_documents

In [2]:
dialogue_list = def_pre(origin)
strip_senten = def_remove_space(dialogue_list)
full_senten = def_remove_space2(strip_senten)

sentence_3 = bind_document(full_senten)
print("발화의 수 : ",len(sentence_3))

#명사 추출
origin_noun_kkma = def_kkmaNoun(sentence_3)
origin_noun_kkma2 = def_remove_space2(origin_noun_kkma)
dictionary_noun = []

for i in range(len(origin_noun_kkma2)):
    for j in range(len(origin_noun_kkma2[i])):
        if origin_noun_kkma2[i][j] not in dictionary_noun:
            dictionary_noun.append(origin_noun_kkma2[i][j])

발화의 수 :  49


In [3]:
# LDA model

from gensim import corpora
import gensim

id2word = corpora.Dictionary(origin_noun_kkma2)
texts = origin_noun_kkma2
corpus = [id2word.doc2bow(text) for text in texts]

lda_model = []

for i in range(3,30):

    lda_model.append(gensim.models.ldamodel.LdaModel(corpus=corpus,
    id2word=id2word,
    num_topics=i,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=50,
    alpha='auto',
    per_word_topics=True))

from gensim.models import CoherenceModel
import numpy as np

Coherence_scores = []
Perplexity_scores = []

for i, lda_model_num in enumerate(lda_model):
    coherence_model_lda = CoherenceModel(model=lda_model_num, texts=origin_noun_kkma2, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print("topic 수 : ", i+3, "  Coherence Score : ", coherence_lda, "  Perplexity : ", np.exp2(-lda_model_num.log_perplexity(corpus)))
    Coherence_scores.append(coherence_lda)
    Perplexity_scores.append(np.exp2(-lda_model_num.log_perplexity(corpus))) 

topic 수 :  3   Coherence Score :  0.360805989639881   Perplexity :  20.62402293874072
topic 수 :  4   Coherence Score :  0.3687651363366631   Perplexity :  19.59981381996606
topic 수 :  5   Coherence Score :  0.34025368286316243   Perplexity :  20.782470812522885
topic 수 :  6   Coherence Score :  0.3809812842751124   Perplexity :  20.44971430736325
topic 수 :  7   Coherence Score :  0.38328528136274376   Perplexity :  19.392491605025462
topic 수 :  8   Coherence Score :  0.39317396569103513   Perplexity :  19.219521709546797
topic 수 :  9   Coherence Score :  0.39318139722829776   Perplexity :  20.339783803379078
topic 수 :  10   Coherence Score :  0.3833526269183719   Perplexity :  19.975597442318495
topic 수 :  11   Coherence Score :  0.4281386243771206   Perplexity :  19.955556714797055
topic 수 :  12   Coherence Score :  0.3927137059198262   Perplexity :  19.096071997603982
topic 수 :  13   Coherence Score :  0.43727522109156136   Perplexity :  19.341158890207435
topic 수 :  14   Coherence S

In [4]:
# Standardization Coherence, Perplexity Score
import numpy as np

co_mean = np.mean(Coherence_scores)
co_std = np.std(Coherence_scores)
per_mean = np.mean(Perplexity_scores)
per_std = np.std(Perplexity_scores)

stan = []

for i in range(len(Coherence_scores)):
    co_standard = (Coherence_scores[i] - co_mean) / co_std
    per_standard = (Perplexity_scores[i] - per_mean) / per_std
    print("토픽 ", i+3, "개   Coherence : ", co_standard, "  Perplexity : ", per_standard, "  co-per : ", co_standard - per_standard)
    stan.append(co_standard - per_standard)

print("최적의 topic 수 : ", stan.index(max(stan))+3)

토픽  3 개   Coherence :  -1.782310790392596   Perplexity :  -0.34831061553916043   co-per :  -1.4340001748534357
토픽  4 개   Coherence :  -1.5189807626287888   Perplexity :  -0.9678688902045455   co-per :  -0.5511118724242433
토픽  5 개   Coherence :  -2.462288144788096   Perplexity :  -0.2524634770084901   co-per :  -2.209824667779606
토픽  6 개   Coherence :  -1.1148069612966398   Perplexity :  -0.4537522450123869   co-per :  -0.6610547162842528
토픽  7 개   Coherence :  -1.0385787373214506   Perplexity :  -1.0932810883149655   co-per :  0.05470235099351495
토픽  8 개   Coherence :  -0.7114095547545191   Perplexity :  -1.1979128829434578   co-per :  0.4865033281889387
토픽  9 개   Coherence :  -0.7111636807948789   Perplexity :  -0.5202510645832671   co-per :  -0.19091261621161182
토픽  10 개   Coherence :  -1.036350595573479   Perplexity :  -0.7405519373363455   co-per :  -0.2957986582371336
토픽  11 개   Coherence :  0.44540346720943486   Perplexity :  -0.7526752174679658   co-per :  1.1980786846774008
토픽 